In [1]:
import time, cv2

import numpy as np
import torch
from torchvision.transforms import Normalize

import utils.config as config
import utils.image_processing as ipc

/home/coffeetumbler/anaconda3/envs/sr_trans/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Denormalize images.
img = torch.randint(256, (4, 3, 5, 5)) / 255
print(img[0])

In [ ]:
img_normalize = Normalize(mean=config.IMG_NORM_MEAN, std=config.IMG_NORM_STD)
normalized_img = img_normalize(img)
print(normalized_img[0])

In [ ]:
denormalized_img = ipc.denormalize(normalized_img)
print(denormalized_img[0])

In [ ]:
# Check runtime for denormalization.
start_time = time.time()
for _ in range(1000):
    ipc.denormalize(normalized_img)
print('runtime :', time.time() - start_time)

In [ ]:
# Quantize images.
noised_img = normalized_img + torch.randn(4, 3, 5, 5) * 0.1
print(noised_img[0])

In [ ]:
noised_denormalized = ipc.denormalize(noised_img)
print(noised_denormalized[0])

In [ ]:
quantized_img = ipc.quantize(noised_denormalized)
print(quantized_img[0])

In [ ]:
print(ipc.quantize(denormalized_img)[0])

In [ ]:
# Y channel conversion
y_channel_pixel = ipc.convert_y_channel(img, add_bias=True)
print(y_channel_pixel[0])

In [ ]:
y_channel_diff = ipc.convert_y_channel(img, add_bias=False)
print(y_channel_diff[0])

In [2]:
# Matlab imresize test
base_dir = '/mnt/d/ubuntu/datasets/DIV2K/DIV2K_swinir/'

hr_img = cv2.imread(base_dir + 'DIV2K_train_HR/0001.png')
lr_img = cv2.imread(base_dir + 'DIV2K_train_LR_bicubic/X2/0001x2.png')

In [3]:
print(hr_img.shape, lr_img.shape)

(1404, 2040, 3) (702, 1020, 3)


In [4]:
lr_img_matlab = ipc.imresize_np(hr_img / 255, 0.5)
print(lr_img_matlab.shape)
lr_img_matlab = np.round(lr_img_matlab * 255)

(702, 1020, 3)


In [9]:
lr_img_cv2 = cv2.resize(hr_img, dsize=(1020, 702), interpolation=cv2.INTER_CUBIC)
print(lr_img_cv2.shape)
lr_img_cv2 = np.round(lr_img_cv2)

(702, 1020, 3)


In [10]:
np.sum(lr_img_matlab != lr_img) / np.product(lr_img.shape)

0.14992737835875092

In [11]:
np.sum(lr_img_cv2 != lr_img) / np.product(lr_img.shape)

0.763410330893991

In [43]:
# Matlab imresize test
# base_dir = 'datasets/Set5/'

# hr_img = cv2.imread(base_dir + 'HR/baby.png')
# lr_img = cv2.imread(base_dir + 'LR_bicubic/X2/babyx2.png')

base_dir = '/mnt/d/ubuntu/datasets/DIV2K/DIV2K_swinir/'

hr_img = cv2.imread(base_dir + 'DIV2K_train_HR/0001.png')
lr_img = cv2.imread(base_dir + 'DIV2K_train_LR_bicubic/X2/0001x2.png')

In [44]:
print(hr_img.shape, lr_img.shape)

(1404, 2040, 3) (702, 1020, 3)


In [47]:
lr_img_cv2 = cv2.resize(hr_img, dsize=(lr_img.shape[1], lr_img.shape[0]), interpolation=cv2.INTER_CUBIC)
print(lr_img_cv2.shape)
lr_img_cv2 = np.round(lr_img_cv2)

(702, 1020, 3)


In [48]:
hr_img = torch.from_numpy(hr_img).permute(2, 0, 1)
lr_img = torch.from_numpy(lr_img).permute(2, 0, 1)

In [49]:
lr_img_matlab = ipc.imresize(hr_img / 255, 0.5)
print(lr_img_matlab.shape)
lr_img_matlab = torch.round(lr_img_matlab * 255)

torch.Size([3, 702, 1020])


In [50]:
torch.sum(lr_img_matlab != lr_img) / np.product(lr_img.shape)

tensor(0.1499)

In [53]:
def PSNR(A, B):
    diff=A-B
    diff = ipc.convert_y_channel(diff, add_bias=False)
    mse = (diff**2).mean(dim=(1,2))
    return (-10)*torch.log10(mse).sum()

print(PSNR(lr_img_matlab / 255, lr_img / 255).item())
print(PSNR(torch.from_numpy(lr_img_cv2).permute(2, 0, 1) / 255, lr_img / 255).item())

60.99903106689453
42.19990539550781
